In [1]:
import numpy as np
from toppra.dracula import RunTopp
import toppra as ta
import toppra.algorithm as algo
from toppra import constraint
from toppra.dracula import RunTopp, V_MAX, A_MAX, _check_waypoints

In [2]:
import re
import ast
import numpy as np

def parse_str_arr(s):
    a = re.sub(r"([^[])\s+([^]])", r"\1, \2", s)
    return np.array(ast.literal_eval(a))

In [10]:
waypts = np.loadtxt('test_waypts_2.txt')
coeff = 0.05
vlim = coeff * np.vstack([-V_MAX, V_MAX]).T
alim = coeff * np.vstack([-A_MAX, A_MAX]).T

In [11]:
min_pair_dist, t_sum = _check_waypoints(waypts, vlim)

In [12]:
t_sum

4.150382220973457

In [16]:
N_samples = waypts.shape[0]
min_pair_dist, t_sum = _check_waypoints(waypts, vlim)
x = np.linspace(0, 0.3 * t_sum, N_samples)  # magic number
path = ta.SplineInterpolator(x, waypts.copy(), bc_type="clamped")
pc_vel = constraint.JointVelocityConstraint(vlim)
pc_acc = constraint.JointAccelerationConstraint(
    alim, discretization_scheme=constraint.DiscretizationType.Interpolation
)
instance = algo.TOPPRA(
    [pc_vel, pc_acc],
    path,
    # gridpoints=gridpoints,
    solver_wrapper="seidel",
)
jnt_traj = instance.compute_trajectory(0, 0)
jnt_traj.cspl.x

array([0.        , 0.06254969, 0.08479917, 0.14550188, 0.20779062,
       0.26255584, 0.31762417, 0.37437734, 0.43143142, 0.48949317,
       0.60766725, 0.66419032, 0.7187898 , 0.77317703, 0.8278153 ,
       0.8842258 , 0.94330766, 1.00492471, 1.06400268, 1.11780643,
       1.17361668, 1.225299  , 1.26497913, 1.32615559, 1.35743538,
       1.39307066, 1.42678003, 1.4601496 , 1.53643994, 1.58576961,
       1.63070458, 1.67079042, 1.71632856, 1.76889085, 1.81748922,
       1.86501555, 1.91839386, 1.97154503, 1.99872731, 2.02635416,
       2.08148229, 2.13495696, 2.18593507, 2.23784549, 2.26587575,
       2.29538944, 2.32542873, 2.3550308 , 2.38404065, 2.41245627,
       2.4402592 , 2.46831962, 2.49904266, 2.53392812, 2.61238753,
       2.78517   , 2.96223107, 3.04654196, 3.1191365 , 3.14935713,
       3.17733388, 3.20540442, 3.23356902, 3.261827  , 3.29017772,
       3.31862219, 3.34703083, 3.37496544, 3.40215839, 3.43040495,
       3.46215571, 3.60451687, 3.63986273, 3.67157976, 3.70154

In [16]:
x_scale, x_max

(4.999999999999999, 2.841646833949905)

array([0.00000000e+00, 3.65152340e-02, 5.37211333e-02, ...,
       1.02156995e+02, 1.02169705e+02, 1.02196690e+02])